In [13]:
from pathlib import Path
import re

import srt
from datasets import load_from_disk, Dataset, Audio
from tqdm.auto import tqdm

from asr_eval.align.parsing import parse_multivariant_string, split_text_into_tokens
from asr_eval.align.data import MultiVariant
from asr_eval.align.recursive import align

In [8]:
# type: ignore

dataset = list(tqdm(
    load_from_disk('tmp/multivariate/multivariant_v1_200')
    .with_format("np")
))
    
def reformat_multivariant(text: str) -> str:
    text = text.replace('{?}', '<*>')
    text = text.replace('MULTI', '<*>')
    text = text.replace('}{', '} {')
    text = text.replace('}- ', '} - ')
    text = re.sub(r'\}-(?=[\w\{])', '} - ', text)
    text = re.sub(r'(?<=[\w\}])-\{', ' - {', text)
    text = text.replace('\u200b', '')
    text = text.replace('\xa0', '')
    text = text.replace('е{', 'е {')
    text = text.replace('ничего X{24', 'ничего {24')
    text = text.replace('{Э+}х', '{Эx|Ээx|Эээх}')
    text = text.replace('О{у+}', '{Оу|Ооу|Оооу|Оуу|Оууу}')
    text = text.replace('Э{м+}', '{Эм|Эмм|Эммм}')
    for x in '.,!?:ЯФ"':
        text = text.replace('}' + x, '} ' + x)
    for x in '"ь':
        text = text.replace(x + '{', x + ' {')
    blocks = parse_multivariant_string(text)  # check that we can parse without errors
    
    for block in blocks[::-1]:
        if isinstance(block, MultiVariant):
            options = text[block.pos[0] + 1:block.pos[1] - 1]
            options = re.split(r'\s*\|\s*', options)
            base = options[0].lower().replace('ё', 'е')
            if all(o.lower().replace('ё', 'е') == base for o in options[1:]):
                best_option = sorted(options, key=lambda x: 'ё' in x.lower())[-1]
                text = text[:block.pos[0]] + best_option + text[block.pos[1]:]
    
    # print(text)
    
    return text

for sample in tqdm(dataset):
    sample['audio']['sampling_rate'] = int(sample['audio']['sr'])
    
    assert 'ru000' in sample['path_audio']
    sample['origin_subset'] = 'ru000'
    sample['origin_id'] = sample['id']
    sample['origin_start_time'] = float(sample['audio']['start_fr'])
    sample['origin_end_time'] = float(sample['audio']['end_fr'])
    
    sample['transcription_segments'] = [
        dict(
            start_time=subtitle.start.total_seconds(),
            end_time=subtitle.end.total_seconds(),
            text=reformat_multivariant(subtitle.content),
        )
        for subtitle in srt.parse(sample['trans'])
    ]
    
    sample['transcription'] = ' '.join(p['text'] for p in sample['transcription_segments'])
    
    del sample['id']
    del sample['audio']['sr']
    del sample['audio']['start_fr']
    del sample['audio']['end_fr']
    del sample['path_audio']
    del sample['path_srt']
    del sample['trans']

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [10]:
Dataset.from_list(dataset).save_to_disk('tmp/updated/multivariant_v1_200')

Saving the dataset (0/5 shards):   0%|          | 0/200 [00:00<?, ? examples/s]

In [9]:
############################

In [4]:
# очень долго выполняется - почему?

dataset = (
    load_from_disk('tmp/updated/multivariant_v1_200')
    # .with_format("np")
    .cast_column('audio', Audio(sampling_rate=16000))
)
dataset

Dataset({
    features: ['audio', 'duration', 'origin_subset', 'origin_id', 'origin_start_time', 'origin_end_time', 'transcription_segments', 'transcription'],
    num_rows: 200
})

In [5]:
dataset.save_to_disk('tmp/updated/multivariant_v1_200_16k')

Saving the dataset (0/3 shards):   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
dataset = load_from_disk('tmp/updated/multivariant_v1_200_16k')

In [15]:
sample = dataset[0]

In [ ]:
from pisets import Pisets

pisets = Pisets(recognizer='openai/whisper-small')
pisets_segments = pisets(sample['audio']['array'])
pisets_prediction = ' '.join(segment.text for segment in pisets_segments)

Device set to use cuda:0
Device set to use cuda:0
/home/oleg/asr-eval/venv/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


'Всем привет, с вами Виктория, и сегодня готовим заварное тесто для эклеров. В сотейнике соединяем {125 | сто двадцать пять} {мл | миллилитров} молока, {125 | сто двадцать пять} {мл | миллилитров} воды, {1 | одну} столовую ложку сахара и {1 | одну} чайную ложку соли. Добавляем {100 | сто} грамм сливочного масла и ставим на небольшой огонь, доводим до закипания и до растворения сливочного масла. Убираем с огня и добавляем сразу всю муку, {150 | сто пятьдесят} грамм. Тщательно перемешиваем, чтобы не осталось никаких комочков. И снова возвращаем ковшик на огонь. Уменьшаем огонь до минимального, регулярно помешиваем и завариваем тесто. У меня на это обычно уходит {3 | три} - {4 | четыре} минуты. Тесто должно собраться в общий комок. Заварную массу перекладываем сразу в отдельную ёмкость , чтобы процесс варки прекратился и тесто немного остыло. И в другую емкость выбиваем {3 | три} яйца. Яйца взбалтываем венчиком либо вилкой и постепенно в несколько раз добавляем яичную массу и перемешиваем

In [18]:
alignment = align(
    parse_multivariant_string(sample['transcription']),
    split_text_into_tokens(pisets_prediction)
)
alignment

MatchesList(matches=[Match(Token(всем), Token(всем)), Match(Token(привет), Token(привет)), Match(Token(с), Token(с)), Match(Token(вами), Token(вами)), Match(Token(виктория), Token(виктория)), Match(Token(и), ), Match(Token(сегодня), Token(сегодня)), Match(Token(готовим), Token(готовим)), Match(, Token(в)), Match(Token(заварное), Token(зверное)), Match(Token(тесто), Token(тесто)), Match(Token(для), Token(для)), Match(Token(эклеров), Token(эклеров)), Match(Token(в), ), Match(Token(сотейнике), Token(сотенники)), Match(Token(соединяем), Token(соединяем)), Match(Token(125), Token(125)), Match(Token(миллилитров), Token(миллилитров)), Match(Token(молока), Token(молока)), Match(Token(125), Token(125)), Match(Token(миллилитров), Token(миллилитров)), Match(Token(воды), Token(воды)), Match(Token(1), Token(1)), Match(Token(столовую), Token(столовую)), Match(Token(ложку), Token(ложку)), Match(Token(сахара), Token(сахара)), Match(Token(и), Token(и)), Match(Token(1), Token(1)), Match(Token(чайную), T

In [19]:
alignment.score.n_word_errors

16